In [18]:
import pandas as pd
import numpy as np
import glob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import os

In [11]:
os.getcwd()

'/home/workspace/DACON/CodeSim/EDA'

In [20]:
val = pd.read_csv("../Dataset/sample_train.csv")

In [21]:
test = pd.read_csv("../Dataset/test.csv")

In [22]:
val.head()

,code1_path,code2_path,code1,code2,similar
0,./train_code/problem393/problem393_19.cpp,./train_code/problem033/problem033_439.cpp,#include <bits/stdc++.h>\n\nusing namespace st...,#include <algorithm>\n#include <bitset>\n#incl...,0
1,./train_code/problem019/problem019_210.cpp,./train_code/problem019/problem019_63.cpp,#include <iostream>\n\nusing namespace std;\n\...,#include <iostream>\n#include <string>\nusing ...,1
2,./train_code/problem107/problem107_486.cpp,./train_code/problem107/problem107_340.cpp,#include <iostream>\n#include <vector>\nusing ...,#include <cstdio>\n#include <cstdlib>\n#includ...,1
3,./train_code/problem187/problem187_257.cpp,./train_code/problem403/problem403_135.cpp,#include <bits/stdc++.h>\n#include <unordered_...,#include <bits/stdc++.h>\nusing namespace std;...,0
4,./train_code/problem173/problem173_490.cpp,./train_code/problem173/problem173_345.cpp,#include <bits/stdc++.h>\ntypedef long long ll...,"#include ""bits/stdc++.h""\n#define rep(i,n) for...",1


In [14]:
test.head()

,pair_id,code1,code2
0,TEST_000000,#include <bits/stdc++.h>\nusing namespace std;...,"#include <bits/stdc++.h>\n#define rep(i, n) fo..."
1,TEST_000001,"#include<bits/stdc++.h>\n#define rep(i,n)for(i...",// //bitset操作\n// #include <iostream>\n// #inc...
2,TEST_000002,#include <bits/stdc++.h>\nusing namespace std;...,#include <bits/stdc++.h>\n#include <ext/pb_ds/...
3,TEST_000003,#include <bits/stdc++.h>\nusing namespace std;...,#include <bits/stdc++.h>\nusing namespace std;...
4,TEST_000004,#include<bits/stdc++.h>\nusing namespace std;\...,#include<iostream>\n#include<algorithm>\n#incl...


In [15]:
len(test)

595000

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import glob
from tqdm import tqdm


class BaselineModel():
    def __init__(self, threshold = 0.5) -> None:
        super(BaselineModel, self).__init__() # 이름 정의하는 것이다. 
        self.threshold = threshold
        self.vocabulary = set()
        self.vectorizer = None

    def get_vectorizer(self) -> CountVectorizer:
        return CountVectorizer(vocabulary=list(self.vocabulary))

    def fit(self, code):
        temp_vectorizer = CountVectorizer()
        temp_vectorizer.fit(code)
        # print(temp_vectorizer.get_feature_names_out())
        self.vocabulary.update(temp_vectorizer.get_feature_names_out())
        self.vectorizer = self.get_vectorizer()
    
    def predict_prob(self, code1, code2) -> np.array:
        code1_vectors = self.vectorizer.transform(code1)
        code2_vectors = self.vectorizer.transform(code2)

        preds = []

        for code1_vec, code2_vec in tqdm(zip(code1_vectors ,code2_vectors)):
            preds.append(cosine_similarity(code1_vec, code2_vec))
        
        preds = np.reshape(preds, len(preds))

        return preds
    
    def predict(self, code1, code2) -> np.array:
        preds = self.predict_proba(code1, code2) # 유사도 계산합니다.
        preds = np.where(preds>self.threshold, 1, 0) # threshold를 기준으로 판별합니다.
        return preds

def read_cpp_code(file_path): 
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def get_accuracy(gt, preds): # 모델의 정확도 측정을 위한 함수입니다.
    return (gt == preds).mean()

In [ ]:
val = pd.read_csv("../Dataset/sample_train.csv")
test = pd.read_csv("../Dataset/test.csv")
train_code_paths = glob.glob('../Dataset/train_code/*/*.cpp')

model = BaselineModel(threshold=0.5)

for path in tqdm(train_code_paths):
    code = read_cpp_code(path)
    model.fit([code]) # 읽어온 코드를 리스트로 묶어 모델에 학습시킵니다

val_preds = model.predict(val['code1'], val['code2'])
print(get_accuracy(val['similar'].values, val_preds))

# 모델 추론
preds = model.predict(test['code1'], test['code2'])

# 제출
submission = pd.read_csv('./sample_submission.csv')
submission['similar'] = preds
submission.to_csv('./base_submit.csv', index=False)